In [1]:
# library imports
import csv
import nltk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# import links file
file = "../Data/ndp_links_all_layers.txt"

In [44]:
# load the NDP plain text as a dataframe
df = pd.read_csv(file, sep = ",", usecols=[0,1,2,3], header=None, error_bad_lines=False, quoting=csv.QUOTE_NONE)
df = df.sort_values(0)
print(df)

                 0                                                  1  \
3428    ((20100803  http://www.ndp.ca/category/story-type/press-re...   
472872  ((20100803  http://www.ndp.ca/print/category/related-conte...   
149732  ((20100803  http://www.ndp.ca/category/related-content/lab...   
2819    ((20100803  http://www.ndp.ca/category/story-type/speeches...   
472299  ((20100803  http://www.ndp.ca/print/category/related-conte...   
147409  ((20100803  http://www.ndp.ca/category/related-content/par...   
144313  ((20100803  http://www.ndp.ca/category/related-content/hum...   
4132    ((20100803   http://www.ndp.ca/category/story-type/fact-check   
2845    ((20100803  http://www.ndp.ca/category/story-type/statemen...   
1921    ((20100803  http://www.ndp.ca/category/story-type/press-re...   
140999  ((20100803          http://www.ndp.ca/print/taxonomy/term/158   
384551  ((20100803  http://www.ndp.ca/print/category/related-conte...   
168571  ((20100803          http://www.ndp.ca/print

In [45]:
# Convert all dates to just month and year as some
# crawls took more than one day

df[0] = df[0].str[6:8] + "/" + df[0].str[4:6]
# Find the different crawl dates
times = df[0].unique()
print(times)

['08/10' '05/11' '08/11' '11/11' '02/12' '05/12' '08/12' '11/12' '02/13'
 '05/13' '08/13' '11/13' '02/14' '05/14' '08/14' '11/14' '02/15' '05/15'
 '11/15']


In [60]:
# Find all the unique pages across all time stamps
pages = (df[1].unique()).tolist()

#print(df[1].unique())
#print(df[2].unique())

print(len(pages))
print(len(df[2].unique()))
pages.extend((df[2].unique()).tolist())
print(len(pages))

pages = set(pages)
print(len(pages))

# This suggests that there are no "linked to" pages that then link back and thus we have a bipartite graph
# https://en.wikipedia.org/wiki/Bipartite_graph

97082
141
97223
97223


In [61]:
# Loop of all different dates
for y in range (0,2):#,len(times)-1):
    # Find each row of data with the correct crawl date
    links_t0 = df.loc[df[0] == times[y]]
    
    # Construct matrix with the webpages down the side and across the top.
    # There is a a 1 if there is a link from the origin webpage in that row to the destination link in that column
    # This matrix completely describes the network
    # Then, there are several metrics that can be used to quantify change of a network